Successfully created 'train_data.csv' and 'test_data.csv'


In [1]:
#  Step 1: Import Libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.filterwarnings("ignore")

#  Step 2: Load Dataset
df = pd.read_csv("output_chunk_4.csv")



In [5]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, ne_chunk

# Download required NLTK resources
nltk.download("all")
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")
nltk.download("averaged_perceptron_tagger")  # POS Tagging
nltk.download("maxent_ne_chunker")  # Named Entity Recognition
nltk.download("words")  # Required for NER
nltk.download('punkt_tab')
# Initialize lemmatizer and stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words("english"))

def preprocess_text(text):
    # 1. Convert to lowercase
    text = text.lower()

    # 2. Remove URLs, emails, and hashtags
    text = re.sub(r"http\S+|www\S+", "", text)  # Remove URLs
    text = re.sub(r"\S+@\S+\.\S+", "", text)  # Remove emails
    text = re.sub(r"#\w+", "", text)  # Remove hashtags

    # 3. Remove special characters and numbers
    text = re.sub(r"[^a-z\s]", "", text)

    # 4. Tokenization
    tokens = word_tokenize(text)

    # 5. Part-of-Speech (POS) Tagging
    pos_tags = pos_tag(tokens)

    # 6. Named Entity Recognition (NER) - Identify named entities
    #named_entities = ne_chunk(pos_tags, binary=True)

    # 7. Stopword removal & Lemmatization
    clean_tokens = [
        lemmatizer.lemmatize(word)
        for word, tag in pos_tags
        if word not in stop_words
    ]

    return " ".join(clean_tokens)

# Apply preprocessing to the dataset
df["clean_text"] = df["text"].apply(preprocess_text)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_eng.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_rus to
[nltk_data]    |     /root

In [8]:
test_df = pd.read_csv("test_data.csv")
test_df.head()

,category,text,clean_text
0,religion,a palestinian protester throws a burning proje...,palestinian protester throw burning projectile...
1,humanInterest,johnny gilbert the voice of jeopardy keeps goi...,johnny gilbert voice jeopardy keep going image...
2,health,a woman walks along a path in a deserted field...,woman walk along path deserted field zvimba ru...
3,education,saratoga springs ny thursday afternoon saratog...,saratoga spring ny thursday afternoon saratoga...
4,social,lifenews a pro life classmate of texas valedic...,lifenews pro life classmate texas valedictoria...


In [9]:
# prompt: Code to load the data and make a stratified split of 2% and save it separately as a csv file . The input file name is output_chunk_4.csv

import pandas as pd
from sklearn.model_selection import train_test_split





# Perform stratified split
train_data, test_data = train_test_split(df, test_size=0.02, stratify=df['category'], random_state=42)

# Save the split data to separate CSV files
try:
    train_data.to_csv('train_data.csv', index=False)
    test_data.to_csv('test_data.csv', index=False)
    print("Successfully created 'train_data.csv' and 'test_data.csv'")
except Exception as e:
    print(f"An error occurred while saving the files: {e}")


Successfully created 'train_data.csv' and 'test_data.csv'


In [10]:
# Define features & labels
X = df["clean_text"]
y = df["category"]  # Assuming 'category' is the label column

# Split into train & validation (keeping test set separate)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Check split sizes
print(f"Train set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(test_df)}")  # Test set remains unchanged


Train set size: 5760
Validation set size: 1440
Test set size: 144


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Convert text to TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 3))

X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_val_tfidf = tfidf_vectorizer.transform(X_val)
X_test_tfidf = tfidf_vectorizer.transform(test_df["text"])  # Apply to test set


In [ ]:
import gensim
from gensim.models import Word2Vec
import numpy as np

# Tokenize text
df["tokens"] = df["clean_text"].apply(lambda x: x.split())
test_df["tokens"] = test_df["text"].apply(lambda x: x.split())

# Train Word2Vec Model
w2v_model = Word2Vec(sentences=df["tokens"], vector_size=100, window=5, min_count=2, workers=4)

# Create sentence embeddings
def vectorize_text(text_tokens, model):
    valid_words = [word for word in text_tokens if word in model.wv]
    if valid_words:
        return np.mean([model.wv[word] for word in valid_words], axis=0)
    else:
        return np.zeros(model.vector_size)  # Handle unseen words

X_train_w2v = np.array(X_train.apply(lambda x: vectorize_text(x.split(), w2v_model)).tolist())
X_val_w2v = np.array(X_val.apply(lambda x: vectorize_text(x.split(), w2v_model)).tolist())
X_test_w2v = np.array(test_df["text"].apply(lambda x: vectorize_text(x.split(), w2v_model)).tolist())


In [13]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize BoW Vectorizer
bow_vectorizer = CountVectorizer(max_features=5000, ngram_range=(1, 3))

# Convert text to BoW vectors
X_train_bow = bow_vectorizer.fit_transform(X_train)
X_val_bow = bow_vectorizer.transform(X_val)
X_test_bow = bow_vectorizer.transform(test_df["text"])


In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

svm_bow = SVC(kernel="linear", probability=True, random_state=42)
svm_bow.fit(X_train_bow, y_train)

# Evaluate
y_pred_bow = svm_bow.predict(X_val_bow)
print("\nBag of Words SVM Performance on Validation:")
print(classification_report(y_val, y_pred_bow))



Bag of Words SVM Performance on Validation:
               precision    recall  f1-score   support

         arts       0.67      0.68      0.68        78
        crime       0.85      0.82      0.84        78
     disaster       0.75      0.73      0.74        78
      economy       0.56      0.59      0.57        78
    education       0.87      0.84      0.85        79
environmental       0.88      0.90      0.89        78
       health       0.77      0.82      0.80        79
humanInterest       0.59      0.67      0.63        79
       labour       0.74      0.82      0.78        78
    lifestyle       0.76      0.87      0.81        79
        other       0.28      0.32      0.30        79
     politics       0.83      0.81      0.82        78
     religion       0.61      0.61      0.61        79
      science       0.84      0.79      0.82        78
       social       0.91      0.76      0.83        79
        sport       0.81      0.71      0.75        78
       unrest      

In [15]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
svm_tfidf = SVC(kernel="poly", probability=True, random_state=42)
svm_tfidf.fit(X_train_tfidf, y_train)

# Evaluate
y_pred_tfidf = svm_tfidf.predict(X_val_tfidf)
print("\nTF-IDF SVM Performance on Validation:")
print(classification_report(y_val, y_pred_tfidf))


KeyboardInterrupt: 

In [17]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
svm_tfidf = SVC(kernel="rbf", probability=True, random_state=42)
svm_tfidf.fit(X_train_tfidf, y_train)

# Evaluate
y_pred_tfidf = svm_tfidf.predict(X_val_tfidf)
print("\nTF-IDF SVM Performance on Validation:")
print(classification_report(y_val, y_pred_tfidf))


TF-IDF SVM Performance on Validation:
               precision    recall  f1-score   support

         arts       0.74      0.86      0.80        80
        crime       0.91      0.78      0.84        80
     disaster       0.76      0.80      0.78        80
      economy       0.76      0.71      0.74        80
    education       0.95      0.86      0.90        80
environmental       0.96      0.85      0.90        80
       health       0.89      0.80      0.84        80
humanInterest       0.83      0.71      0.77        80
       labour       0.86      0.85      0.86        80
    lifestyle       0.74      0.78      0.76        80
        other       0.25      0.47      0.33        80
     politics       0.98      0.81      0.89        80
     religion       0.60      0.47      0.53        80
      science       0.81      0.82      0.82        80
       social       0.95      0.68      0.79        80
        sport       0.72      0.85      0.78        80
       unrest       0.78 

In [16]:
#import random forest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
rf_bow = RandomForestClassifier(n_estimators=100, random_state=42)
# train with tfidf vectorised data
rf_bow.fit(X_train_bow, y_train)

# Evaluate
y_pred_bow = rf_bow.predict(X_val_bow)
print("\nBag of Words Random Forest Performance on Validation:")
print(classification_report(y_val, y_pred_bow))



Bag of Words Random Forest Performance on Validation:
               precision    recall  f1-score   support

         arts       0.81      0.91      0.86        80
        crime       0.73      0.76      0.74        80
     disaster       0.76      0.90      0.82        80
      economy       0.62      0.49      0.55        80
    education       0.70      0.89      0.78        80
environmental       0.83      0.86      0.85        80
       health       0.81      0.82      0.82        80
humanInterest       0.58      0.55      0.56        80
       labour       0.76      0.80      0.78        80
    lifestyle       0.77      0.89      0.83        80
        other       0.30      0.07      0.12        80
     politics       0.87      0.72      0.79        80
     religion       0.86      0.68      0.76        80
      science       0.72      0.89      0.80        80
       social       0.85      0.56      0.68        80
        sport       0.67      0.88      0.76        80
       un

In [ ]:
svm_w2v = SVC(kernel="linear", probability=True, random_state=42)
svm_w2v.fit(X_train_w2v, y_train)

# Evaluate
y_pred_w2v = svm_w2v.predict(X_val_w2v)
print("\nWord2Vec SVM Performance on Validation:")
print(classification_report(y_val, y_pred_w2v))



Word2Vec SVM Performance on Validation:
               precision    recall  f1-score   support

         arts       0.51      0.62      0.55        78
        crime       0.72      0.62      0.66        78
     disaster       0.68      0.68      0.68        78
      economy       0.49      0.64      0.55        78
    education       0.84      0.77      0.80        79
environmental       0.86      0.86      0.86        78
       health       0.65      0.78      0.71        79
humanInterest       0.40      0.39      0.39        79
       labour       0.75      0.74      0.75        78
    lifestyle       0.74      0.77      0.76        79
        other       0.25      0.13      0.17        79
     politics       0.56      0.62      0.59        78
     religion       0.47      0.46      0.46        79
      science       0.73      0.79      0.76        78
       social       0.74      0.63      0.68        79
        sport       0.78      0.82      0.80        78
       unrest       0.7

In [ ]:
# Predict on Test Set
y_test_pred_bow = svm_bow.predict(X_test_bow)
y_test_pred_tfidf = svm_tfidf.predict(X_test_tfidf)
y_test_pred_w2v = svm_w2v.predict(X_test_w2v)

# Evaluate
print("\nFinal Test Performance (Bag of Words SVM):")
print(classification_report(test_df["category"], y_test_pred_bow))

print("\nFinal Test Performance (TF-IDF SVM):")
print(classification_report(test_df["category"], y_test_pred_tfidf))

print("\nFinal Test Performance (Word2Vec SVM):")
print(classification_report(test_df["category"], y_test_pred_w2v))



Final Test Performance (Bag of Words SVM):
               precision    recall  f1-score   support

         arts       0.67      0.75      0.71         8
        crime       0.70      0.88      0.78         8
     disaster       0.64      0.88      0.74         8
      economy       0.83      0.62      0.71         8
    education       1.00      1.00      1.00         8
environmental       1.00      0.75      0.86         8
       health       0.75      0.75      0.75         8
humanInterest       0.78      0.88      0.82         8
       labour       0.67      0.25      0.36         8
    lifestyle       0.50      0.88      0.64         8
        other       0.25      0.38      0.30         8
     politics       1.00      0.75      0.86         8
     religion       0.43      0.38      0.40         8
      science       0.86      0.75      0.80         8
       social       1.00      0.75      0.86         8
        sport       1.00      0.88      0.93         8
       unrest       

In [ ]:
import joblib

# Save the trained SVM model
joblib.dump(svm_tfidf, "svm_tfidf_model.pkl")

# Save the TF-IDF vectorizer
joblib.dump(tfidf_vectorizer, "tfidf_vectorizer.pkl")

print("SVM model and TF-IDF vectorizer saved successfully!")


SVM model and TF-IDF vectorizer saved successfully!
